In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.4 MB/s eta 0:00:

In [ ]:
!nvidia-smi

Tue Apr 25 13:55:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import pipeline, set_seed
from transformers import BartTokenizer, BartModel # bart

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from rouge_score import rouge_scorer 

In [ ]:
from transformers import AutoTokenizer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from transformers import BartForConditionalGeneration

In [ ]:
model_large = "facebook/bart-large-cnn" # bart base

#tokenizer_large = BartTokenizer.from_pretrained(model_large)
tokenizer_large =  AutoTokenizer.from_pretrained(model_large)
#model_large = BartModel.from_pretrained(model_large).to(device)
model_large = BartForConditionalGeneration.from_pretrained(model_large).to(device)

In [ ]:
import numpy as np
import pandas as pd
import json

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

# LoadData

In [ ]:
dataset_xsum = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_samsum = dataset_xsum

In [ ]:
gpt_load = np.load('/content/samsum_ChatGPT_full-raoyi.npy')

In [ ]:
len(gpt_load)

14732

In [ ]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
dataset_samsum["train"]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
type(dataset_samsum["train"]["id"])

list

In [ ]:
df = pd.DataFrame (dataset_samsum["train"]["id"], columns = ['id'])

In [ ]:
df_test = pd.DataFrame (dataset_samsum["test"]["id"], columns = ['id'])

In [ ]:
df['dialogue'] = dataset_samsum["train"]["dialogue"]

In [ ]:
df_test['dialogue'] = pd.DataFrame (dataset_samsum["test"]["dialogue"])

In [ ]:
df['samsum'] = dataset_samsum["train"]["summary"]

In [ ]:
df_test['summary'] = pd.DataFrame (dataset_samsum["test"]["summary"])

In [ ]:
gpt_list = gpt_load.tolist()

In [ ]:
df['gpt'] = gpt_list

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
df.index

RangeIndex(start=0, stop=14732, step=1)

In [ ]:
df

,dialogue,samsum,gpt
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...,"Amanda offered Jerry some cookies, and he acce..."
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...,Olivia and Oliver both plan to vote for the Li...
2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...,Kim was feeling bad after spending her day pro...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...,Rachel abruptly interrupted Edward's confessio...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com...","Sam overheard his roommate, Rick, talking on t..."
...,...,...,...
14727,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...,Romeo reached out to Greta as a potential frie...
14728,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...,"Theresa shared photos of her workplace, and me..."
14729,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...,John expressed his outrage over Japan's decisi...
14730,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...,Jennifer and Celia catch up on their social li...


In [ ]:
type(df.index[0])

int

In [ ]:
type(df.samsum[0])

str

In [ ]:
df = df.reset_index()

In [ ]:
df = df.astype({"index": int})

In [ ]:
df = df.rename(columns={"index": "id"})

In [ ]:
def assign_sum(dff):
  if int(dff.id) % 2 == 0:
    return str(dff.samsum) 
  else:
    return str(dff.gpt)

In [ ]:
df["summary"] = df.apply(lambda x:assign_sum(x), axis=1)

In [ ]:
df.to_csv('/content/processed_df.csv', sep='\t', index=False)

In [ ]:
df.to_json(r'/content/json_Processed_df.txt')

In [ ]:
df.head()

,id,dialogue,samsum,gpt,summary
0,0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...,"Amanda offered Jerry some cookies, and he acce...",Amanda baked cookies and will bring Jerry some...
1,1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...,Olivia and Oliver both plan to vote for the Li...,Olivia and Oliver both plan to vote for the Li...
2,2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...,Kim was feeling bad after spending her day pro...,Kim may try the pomodoro technique recommended...
3,3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...,Rachel abruptly interrupted Edward's confessio...,Rachel abruptly interrupted Edward's confessio...
4,4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com...","Sam overheard his roommate, Rick, talking on t...","Sam is confused, because he overheard Rick com..."


In [ ]:
df_test

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....
...,...,...,...
814,13611902-1,Alex: Were you able to attend Friday night's b...,Benjamin didn't come to see a basketball game ...
815,13820989,Jamilla: remember that the audition starts at ...,The audition starts at 7.30 P.M. in Antena 3.
816,13717193,"Marta: <file_gif>\r\nMarta: Sorry girls, I cli...","Marta sent a file accidentally,"
817,13829115,Cora: Have you heard how much fuss British med...,There was a meet-and-greet with James Charles ...


In [ ]:
class samsum_test(torch.utils.data.Dataset):
    def __init__(self, df):
        self.data = df

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.data.iloc[idx].dialogue	, self.data.iloc[idx].summary

In [ ]:
test_data = samsum_test(df_test)

In [ ]:
for x, y in test_data:
  print(x)
  print(y)
  break

Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer_large = BartTokenizer.from_pretrained("/content/drive/MyDrive/Project/dl-gp/tokenizer")
model_large = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Project/dl-gp/bart-large-model").to(device)

In [ ]:
from tqdm import tqdm
results = {'rouge1': np.zeros(3), 'rouge2': np.zeros(3), 'rougeL': np.zeros(3)}

for data in tqdm(test_data,position=0, leave=True):
  article, ref = data

  inputs = tokenizer_large(article, return_tensors="pt", max_length=1024).to(device)
  summary_ids = model_large.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=100)
  
  res = tokenizer_large.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL', 'rouge2'], use_stemmer=True)
  scores = scorer.score(res, ref)
  for key in scores.keys():
    results[key] += np.array(scores[key])



100%|██████████| 819/819 [07:20<00:00,  1.86it/s]


In [ ]:
# average over all test data
for key in results.keys(): 
  results[key] /= len(test_data)
results

{'rouge1': array([0.55195888, 0.52645783, 0.50188285]),
 'rouge2': array([0.27909943, 0.27393476, 0.25766099]),
 'rougeL': array([0.45640011, 0.43782755, 0.41680987])}

In [ ]:
# Print Average F1 metrics 
print("average R1: ", results['rouge1'][2])
print("average R2: ", results['rouge2'][2])
print("average RL: ", results['rougeL'][2])

average R1:  0.5018828490298214
average R2:  0.257660989494259
average RL:  0.41680986972473816


In [ ]:
# bertscore

In [ ]:
df_test

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....
...,...,...,...
814,13611902-1,Alex: Were you able to attend Friday night's b...,Benjamin didn't come to see a basketball game ...
815,13820989,Jamilla: remember that the audition starts at ...,The audition starts at 7.30 P.M. in Antena 3.
816,13717193,"Marta: <file_gif>\r\nMarta: Sorry girls, I cli...","Marta sent a file accidentally,"
817,13829115,Cora: Have you heard how much fuss British med...,There was a meet-and-greet with James Charles ...


In [ ]:
test_dialogue_list = df_test.dialogue.to_list()

In [ ]:
len(test_dialogue_list)

819

In [ ]:
test_summary_list = df_test.summary.to_list()

In [ ]:
len(test_summary_list)

819

In [ ]:
sample_text = test_dialogue_list[760]
reference = test_summary_list[760]

In [ ]:
name = 'philschmid/bart-large-cnn-samsum'
tokenizer_large_baseline = BartTokenizer.from_pretrained(name)
model_large_baseline = BartForConditionalGeneration.from_pretrained(name).to(device)


In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
pipe = pipeline("summarization", model='/content/drive/MyDrive/Project/dl-gp/bart-large-model',tokenizer=tokenizer_large)

In [ ]:
gen_kwargs_baseline = {"length_penalty": 0.6, "num_beams":8, "max_length": 128}
pipe_baseline = pipeline("summarization", model=name,tokenizer=tokenizer_large_baseline)

In [ ]:
outputs_all_samsum = []
outputs_all_gpt = []

In [ ]:
for i in range(len(test_summary_list)):
  if i % 50 == 0:
    print(str(i))
  sample_text = test_dialogue_list[i]
  #reference = test_summary_list[i]

  out_gpt = pipe(sample_text, **gen_kwargs)[0]["summary_text"]
  outputs_all_gpt.append(out_gpt)

  out_samsum = pipe_baseline(sample_text, **gen_kwargs_baseline)[0]["summary_text"]
  outputs_all_samsum.append(out_samsum)

In [ ]:
# baseline 
name_no_tune = 'facebook/bart-large-cnn'
tokenizer_large_no_tune = BartTokenizer.from_pretrained(name_no_tune)
model_large_no_tune = BartForConditionalGeneration.from_pretrained(name_no_tune).to(device)


In [ ]:
gen_kwargs_no_tune = {"length_penalty": 0.6, "num_beams":8, "max_length": 128}
pipe_no_tune = pipeline("summarization", model=name_no_tune,tokenizer=tokenizer_large_no_tune)

In [ ]:
baselin_out = []

In [ ]:
len(test_summary_list)

819

In [ ]:
len(test_dialogue_list)

819

In [ ]:
for i in range(len(test_summary_list)):
  if i % 50 == 0:
    print(str(i))
  sample_text = test_dialogue_list[i]

  out_base = pipe_no_tune(sample_text, **gen_kwargs_no_tune)[0]["summary_text"]
  baselin_out.append(out_base)


In [ ]:
len(test_summary_list)

819

In [ ]:
with open(r'/content/outputs_baseline.txt', 'w') as fp:
    for item in baselin_out:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
predictions_base = baselin_out
references = test_summary_list
results_gpt = bertscore.compute(predictions=predictions_base, references=references, lang="en")
pd.DataFrame(results_gpt).describe()

,precision,recall,f1
count,819.000000,819.000000,819.000000
mean,0.848034,0.896337,0.871295
std,0.026192,0.021211,0.019706
min,0.709627,0.823914,0.800108
25%,0.831008,0.882184,0.858547
50%,0.847193,0.896325,0.870559
75%,0.864721,0.910897,0.883709
max,0.925989,0.957649,0.941553


In [ ]:
half_path = '/content/drive/MyDrive/Project/dl-gp/bart-half-3'
tokenizer_half = BartTokenizer.from_pretrained(half_path)
model_large_half= BartForConditionalGeneration.from_pretrained(half_path).to(device)


In [ ]:
gen_kwargs_half = {"length_penalty": 0.6, "num_beams":8, "max_length": 128}
pipe_half = pipeline("summarization", model=half_path,tokenizer=tokenizer_half)

In [ ]:
print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary GPT:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])
print("\nModel Summary Half:")
print(pipe_half(sample_text, **gen_kwargs_half)[0]["summary_text"])
print("\nModel Summary Baseline:")
print(pipe_baseline(sample_text, **gen_kwargs_baseline)[0]["summary_text"])


Your max_length is set to 128, but you input_length is only 78. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Dialogue:
Pamela: Have you tried applying for the RGS grant?
Zoe: I did last year.
Zoe: Wasn't lucky...
Xiara: Me neither
Pamela: Do you know if I have to be a member?
Zoe: It depends on the award
Pamela: Ok I'll have a look at their website 

Reference Summary:
Pamela will look at their website to apply for the RGS grant. Zoe and Xiara applied last year but they weren't lucky. Sometimes you have to be a member to apply.

Model Summary GPT:


Your max_length is set to 128, but you input_length is only 78. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Pamela asks Zoe and Xiara if they tried applying for the RGS grant last year, but Zoe didn't get lucky. Pamela asks if she has to be a member, and Zoe says it depends on the award. Pamela decides to check the website.

Model Summary Half:


Your max_length is set to 128, but you input_length is only 78. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Zoe and Xiara haven't tried applying for the RGS grant last year. Pamela wants to know if she has to be a member, but Zoe doesn't know if it depends on the award. Pamela will look at their website for more information. 

Model Summary Baseline:
Zoe and Xiara didn't apply for the RGS grant last year. Pamela will check the website to see if she has to be a member of RGS to be eligible for the grant, but Zoe doesn't know if she needs to be. 


In [ ]:
outputs_half = []

In [ ]:
for i in range(len(test_summary_list)):
  if i % 50 == 0:
    print(str(i))
  sample_text = test_dialogue_list[i]
  #reference = test_summary_list[i]

  out_half = pipe_half(sample_text, **gen_kwargs_half)[0]["summary_text"]
  outputs_half.append(out_half)

In [ ]:
len(test_dialogue_list)

819

In [ ]:
len(outputs_all_samsum)

819

In [ ]:
len(outputs_all_gpt)

819

In [ ]:
len(outputs_half)

819

In [ ]:
with open(r'/content/outputs_half.txt', 'w') as fp:
    for item in outputs_half:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
check_list = []
with open(r'/content/outputs_half.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        # add current item to the list
        check_list.append(x)

In [ ]:
check_list[-1]

'Rachel shared a list of the Top 50 Best Films of 2018 with Janice, who has watched almost all of them. They joked about Deadpool 2 and Avengers, and Janice revealed that her boyfriend forced her to watch it. Rachel complimented Janice on being the best girlfriend in the world.'

In [ ]:
outputs_all_samsum[-1]

"Rachel has sent Janice a list of the Top 50 Best Films of 2018. Janice has watched almost all of them. She has watched Deadpool 2 twice and Avengers twice. Her boyfriend forced her to watch Deadpool 2, but it wasn't that bad. "

In [ ]:
len(check_list)

819

# BertScore

In [ ]:
!pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load
bertscore = load("bertscore")

In [ ]:
test_summary_list[-1]

'Rachel sends a list of Top 50 films of 2018. Janice watched almost half of them, Deadpool 2 and Avengers included.'

In [ ]:
outputs_all_gpt[-1]

"Rachel sent Janice a list of the Top 50 Best Films of 2018. Janice has watched almost all of them, including Deadpool 2 and Avengers 2. Her boyfriend forced her to watch Deadpool 2, but it wasn't as bad as she thought it would be."

In [ ]:
predictions_all_gpt = outputs_all_gpt
references = test_summary_list
results_all_gpt = bertscore.compute(predictions=predictions_all_gpt, references=references, lang="en")

In [ ]:
print(results_all_gpt)

{'precision': [0.8803857564926147, 0.8728463053703308, 0.9108774065971375, 0.8402272462844849, 0.8865672945976257, 0.9109383821487427, 0.8941929340362549, 0.8818567395210266, 0.8953697681427002, 0.8998484015464783, 0.8996928930282593, 0.9041759371757507, 0.8869274258613586, 0.8710382580757141, 0.8929308652877808, 0.8884423971176147, 0.8797466158866882, 0.889814019203186, 0.8552783727645874, 0.8889757394790649, 0.9000822901725769, 0.8898827433586121, 0.9217490553855896, 0.8870884776115417, 0.9299376606941223, 0.8727092146873474, 0.8373060822486877, 0.8894860744476318, 0.9096322059631348, 0.8975367546081543, 0.8684390783309937, 0.8637007474899292, 0.8565617799758911, 0.8920915126800537, 0.9137641191482544, 0.8438608646392822, 0.8519838452339172, 0.8435507416725159, 0.856401801109314, 0.9280562400817871, 0.8597925901412964, 0.8538830280303955, 0.9225724339485168, 0.8476893901824951, 0.882188081741333, 0.8909103870391846, 0.9268420934677124, 0.8617808818817139, 0.8591969013214111, 0.869293

In [ ]:
with open("/content/bertscore_all_gpt.json", "w") as outfile:
    json.dump(results_all_gpt, outfile)

In [ ]:
pd.DataFrame(results_all_gpt).describe()

,precision,recall,f1
count,819.000000,819.000000,819.000000
mean,0.880252,0.921595,0.900279
std,0.024260,0.020153,0.018760
min,0.806842,0.861792,0.837919
25%,0.862506,0.908027,0.888242
50%,0.880343,0.923544,0.900528
75%,0.896824,0.935189,0.912441
max,0.946946,0.974687,0.953862


In [ ]:
predictions_all_samsum = outputs_all_samsum
references = test_summary_list
results_all_samsum = bertscore.compute(predictions=predictions_all_samsum, references=references, lang="en")

In [ ]:
print(results_all_samsum)

{'precision': [0.8439989686012268, 0.8712958097457886, 0.8830504417419434, 0.8492715358734131, 0.9011852741241455, 0.9016435742378235, 0.890662670135498, 0.8646018505096436, 0.8849481344223022, 0.903106689453125, 0.8981284499168396, 0.9096364974975586, 0.9071511626243591, 0.8823152780532837, 0.8841474056243896, 0.8932774662971497, 0.8711445331573486, 0.9076710939407349, 0.8378878235816956, 0.9035516381263733, 0.906855583190918, 0.8875408172607422, 0.9068043231964111, 0.8977020978927612, 0.9399736523628235, 0.8683890104293823, 0.828180193901062, 0.8939269781112671, 0.9123910665512085, 0.8954557180404663, 0.8593969345092773, 0.855899453163147, 0.8635988235473633, 0.8946559429168701, 0.9270034432411194, 0.8482162356376648, 0.8714333772659302, 0.8475374579429626, 0.8492249250411987, 0.901163637638092, 0.8668437004089355, 0.8572643995285034, 0.9020910263061523, 0.8527729511260986, 0.8857295513153076, 0.884060800075531, 0.8969956040382385, 0.855279803276062, 0.8590085506439209, 0.86163246631

In [ ]:
with open("/content/bertscore_all_samsum.json", "w") as outfile:
    json.dump(results_all_samsum, outfile)

In [ ]:
pd.DataFrame(results_all_samsum).describe()

,precision,recall,f1
count,819.000000,819.000000,819.000000
mean,0.881976,0.923330,0.902004
std,0.024608,0.020759,0.019165
min,0.814543,0.834360,0.840206
25%,0.864352,0.908875,0.889396
50%,0.881228,0.923892,0.901497
75%,0.898492,0.937808,0.914577
max,0.968500,0.970951,0.964015


In [ ]:
predictions_half = outputs_half
references = test_summary_list
results_half = bertscore.compute(predictions=predictions_half, references=references, lang="en")

In [ ]:
print(results_half)

{'precision': [0.8805841207504272, 0.8512194752693176, 0.8974968194961548, 0.8356216549873352, 0.8862998485565186, 0.8973331451416016, 0.898887038230896, 0.8557054996490479, 0.8952789902687073, 0.8955339193344116, 0.9015289545059204, 0.901426374912262, 0.8994163274765015, 0.8528918623924255, 0.8971614241600037, 0.8869397640228271, 0.8734956979751587, 0.8972720503807068, 0.8292332887649536, 0.9055600762367249, 0.8919569253921509, 0.893785834312439, 0.8695282340049744, 0.8791254162788391, 0.9317930340766907, 0.8740653991699219, 0.8429074287414551, 0.8870065212249756, 0.9133815765380859, 0.9221428036689758, 0.869624137878418, 0.8502002358436584, 0.8634780645370483, 0.8960505723953247, 0.9100989103317261, 0.8500052094459534, 0.8723288774490356, 0.8526060581207275, 0.8512500524520874, 0.8956254124641418, 0.8638017177581787, 0.840796172618866, 0.8927794694900513, 0.8610203266143799, 0.8822290301322937, 0.8588610291481018, 0.8936536312103271, 0.8649277687072754, 0.8334120512008667, 0.84518623

In [ ]:
with open("/content/bertscore_half.json", "w") as outfile:
    json.dump(results_half, outfile)

In [ ]:
pd.DataFrame(results_half).describe()

,precision,recall,f1
count,819.000000,819.000000,819.000000
mean,0.876887,0.920698,0.898081
std,0.025581,0.019598,0.019316
min,0.766338,0.861125,0.825987
25%,0.860544,0.907443,0.885599
50%,0.877723,0.921563,0.899099
75%,0.894449,0.935335,0.911299
max,0.952414,0.966707,0.951426


# dialogsum Test

In [ ]:
dataset_dia = load_dataset("knkarthick/dialogsum")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_dia

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [ ]:
newtest = pd.DataFrame (dataset_dia["test"]["id"], columns = ['id'])
newtest['dialogue'] = dataset_dia["test"]["dialogue"]
newtest['summary'] = dataset_dia["test"]["summary"]

In [ ]:
newtest = newtest.iloc[200:700]

In [ ]:
len(newtest)

500

In [ ]:
newtest

,id,dialogue,summary
200,test_66_3,#Person1#: Have you considered upgrading your ...,#Person1# teaches #Person2# how to upgrade sof...
201,test_67_1,"#Person1#: Where to, miss?\n#Person2#: Hi! Cre...",#Person1# is driving #Person2# to an inn. They...
202,test_67_2,"#Person1#: Where to, miss?\n#Person2#: Hi! Cre...",#Person1# drives #Person2# to an inn and they ...
203,test_67_3,"#Person1#: Where to, miss?\n#Person2#: Hi! Cre...",#Person1# drives #Person2# from the airport to...
204,test_68_1,"#Person1#: Oh dear, my weight has gone up agai...",#Person1# wants to lose weight. #Person2# sugg...
...,...,...,...
695,test_231_3,"#Person1#: Hi, Laura. Where are you heading wi...",Laura exercises both for health and to look go...
696,test_232_1,#Person1#: Were you born in Los Angeles?\n#Per...,#Person1# asks #Person2# about past experience...
697,test_232_2,#Person1#: Were you born in Los Angeles?\n#Per...,#Person1# is asking #Person2# about the places...
698,test_232_3,#Person1#: Were you born in Los Angeles?\n#Per...,#Person1# and #Person2# talk about #Person2#'s...


In [ ]:
class newtest_test(torch.utils.data.Dataset):
    def __init__(self, df):
        self.data = df

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.data.iloc[idx].dialogue , self.data.iloc[idx].summary

In [ ]:
newtest_loader = newtest_test(newtest)

In [ ]:
for x, y in newtest_loader:
  print(x)
  print(y)
  break

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.


##### rouge

In [ ]:
tokenizer_large = BartTokenizer.from_pretrained("/content/drive/MyDrive/Project/dl-gp/tokenizer")
model_large = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Project/dl-gp/bart-large-model").to(device)

In [ ]:
name_base = 'philschmid/bart-large-cnn-samsum'
tokenizer_large = BartTokenizer.from_pretrained(name_base)
model_large = BartForConditionalGeneration.from_pretrained(name_base).to(device)

In [ ]:
name_half = '/content/drive/MyDrive/Project/dl-gp/bart-half-3'
tokenizer_large = BartTokenizer.from_pretrained(name_half)
model_large = BartForConditionalGeneration.from_pretrained(name_half).to(device)

In [ ]:
results = {'rouge1': np.zeros(3), 'rouge2': np.zeros(3), 'rougeL': np.zeros(3)}

for data in tqdm(newtest_loader,position=0, leave=True):
  article, ref = data

  inputs = tokenizer_large(article, return_tensors="pt", max_length=1024).to(device)
  summary_ids = model_large.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=100)
  
  res = tokenizer_large.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL', 'rouge2'], use_stemmer=True)
  scores = scorer.score(res, ref)
  for key in scores.keys():
    results[key] += np.array(scores[key])



100%|██████████| 500/500 [07:41<00:00,  1.08it/s]


In [ ]:
# average over all test data
for key in results.keys(): 
  results[key] /= len(test_data)
results

{'rouge1': array([0.33994091, 0.18447895, 0.22818755]),
 'rouge2': array([0.13704928, 0.07378643, 0.09125379]),
 'rougeL': array([0.26795599, 0.14426502, 0.17888759])}

In [ ]:
# Print Average F1 metrics 
print("average R1: ", results['rouge1'][2])
print("average R2: ", results['rouge2'][2])
print("average RL: ", results['rougeL'][2])

average R1:  0.22818755310680386
average R2:  0.09125379336492445
average RL:  0.17888759433349882


In [ ]:
# all following values should be *819/500

In [ ]:
'''
all gpt
average R1:  0.23589284340278896
average R2:  0.09249399524623757
average RL:  0.18861802104455763

'''

In [ ]:
'''
all samsum
average R1:  0.24275468763338118
average R2:  0.10179598349934946
average RL:  0.19249650711464608

'''

In [ ]:
'''
samsum gpt half
average R1:  0.22818755310680386
average R2:  0.09125379336492445
average RL:  0.17888759433349882

'''

#### bertscore

In [ ]:
# all chatgpt
tokenizer_gpt = BartTokenizer.from_pretrained('/content/drive/MyDrive/Project/dl-gp/BART-large-full-GPT/bart-tokenizer-3epoch')
gen_kwargs_gpt = {"length_penalty": 0.6, "num_beams":8, "max_length": 128}
pipe_gpt = pipeline("summarization", model='/content/drive/MyDrive/Project/dl-gp/BART-large-full-GPT/bart-large-model-3epoch',tokenizer=tokenizer_gpt)

In [ ]:
# all samsum
name_samsum = 'philschmid/bart-large-cnn-samsum'
tokenizer_samsum = BartTokenizer.from_pretrained(name_samsum)
gen_kwargs_samsum = {"length_penalty": 0.6, "num_beams":8, "max_length": 128}
pipe_samsum = pipeline("summarization", model=name_samsum,tokenizer=tokenizer_samsum)

In [ ]:
# half
name_half = '/content/drive/MyDrive/Project/dl-gp/bart-half-3'
tokenizer_half = BartTokenizer.from_pretrained(name_half)
gen_kwargs_half = {"length_penalty": 0.6, "num_beams":8, "max_length": 128}
pipe_half = pipeline("summarization", model=name_half,tokenizer=tokenizer_half)

In [ ]:
newtest

,id,dialogue,summary
200,test_66_3,#Person1#: Have you considered upgrading your ...,#Person1# teaches #Person2# how to upgrade sof...
201,test_67_1,"#Person1#: Where to, miss?\n#Person2#: Hi! Cre...",#Person1# is driving #Person2# to an inn. They...
202,test_67_2,"#Person1#: Where to, miss?\n#Person2#: Hi! Cre...",#Person1# drives #Person2# to an inn and they ...
203,test_67_3,"#Person1#: Where to, miss?\n#Person2#: Hi! Cre...",#Person1# drives #Person2# from the airport to...
204,test_68_1,"#Person1#: Oh dear, my weight has gone up agai...",#Person1# wants to lose weight. #Person2# sugg...
...,...,...,...
695,test_231_3,"#Person1#: Hi, Laura. Where are you heading wi...",Laura exercises both for health and to look go...
696,test_232_1,#Person1#: Were you born in Los Angeles?\n#Per...,#Person1# asks #Person2# about past experience...
697,test_232_2,#Person1#: Were you born in Los Angeles?\n#Per...,#Person1# is asking #Person2# about the places...
698,test_232_3,#Person1#: Were you born in Los Angeles?\n#Per...,#Person1# and #Person2# talk about #Person2#'s...


In [ ]:
newtest_summary_list = newtest.summary.to_list()
newtest_dialogue_list = newtest.dialogue.to_list()

In [ ]:
len(newtest_summary_list)

500

In [ ]:
len(newtest_dialogue_list)

500

In [ ]:
sample_text = newtest_dialogue_list[200]
reference = newtest_summary_list[200]

In [ ]:
gpt_list = []
samsum_list = []
half_list = []

In [ ]:
for i in range(len(newtest_dialogue_list)):
  if i % 50 == 0:
    print(str(i))
  sample_text = newtest_dialogue_list[i]

  out_gpt = pipe_gpt(sample_text, **gen_kwargs_gpt)[0]["summary_text"]
  gpt_list.append(out_gpt)

  out_half = pipe_half(sample_text, **gen_kwargs_half)[0]["summary_text"]
  half_list.append(out_half)

  out_samsum = pipe_samsum(sample_text, **gen_kwargs_samsum)[0]["summary_text"]
  samsum_list.append(out_samsum)


In [ ]:
with open(r'/content/newtest_gpt.txt', 'w') as fp:
    for item in gpt_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
with open(r'/content/newtest_samsum.txt', 'w') as fp:
    for item in samsum_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
with open(r'/content/newtest_half.txt', 'w') as fp:
    for item in half_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
predictions_gpt = gpt_list
references = newtest_summary_list
results_gpt = bertscore.compute(predictions=predictions_gpt, references=references, lang="en")
pd.DataFrame(results_gpt).describe()

,precision,recall,f1
count,500.000000,500.000000,500.000000
mean,0.878447,0.912362,0.894951
std,0.021399,0.022699,0.019218
min,0.822990,0.833746,0.839351
25%,0.862992,0.899938,0.882509
50%,0.881271,0.914384,0.896300
75%,0.893303,0.928616,0.907713
max,0.933218,0.967221,0.944993


In [ ]:
predictions_samsum = samsum_list
references = newtest_summary_list
results_samsum = bertscore.compute(predictions=predictions_samsum, references=references, lang="en")
pd.DataFrame(results_samsum).describe()

,precision,recall,f1
count,500.000000,500.000000,500.000000
mean,0.879131,0.912465,0.895374
std,0.022695,0.022249,0.020164
min,0.811855,0.844266,0.838776
25%,0.863384,0.899200,0.882649
50%,0.881220,0.913484,0.896171
75%,0.895472,0.929290,0.909640
max,0.935046,0.965356,0.940229


In [ ]:
predictions_half = half_list
references = newtest_summary_list
results_half = bertscore.compute(predictions=predictions_half, references=references, lang="en")
pd.DataFrame(results_half).describe()

,precision,recall,f1
count,500.000000,500.000000,500.000000
mean,0.877313,0.909483,0.892996
std,0.021311,0.021557,0.018986
min,0.796319,0.839132,0.826866
25%,0.864594,0.895475,0.881116
50%,0.876440,0.910310,0.893229
75%,0.893212,0.925214,0.905889
max,0.928801,0.966718,0.939535


In [ ]:
print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary GPT:")
print(pipe_gpt(sample_text, **gen_kwargs_gpt)[0]["summary_text"])
print("\nModel Summary Half:")
print(pipe_half(sample_text, **gen_kwargs_half)[0]["summary_text"])
print("\nModel Summary Samsum:")
print(pipe_samsum(sample_text, **gen_kwargs_samsum)[0]["summary_text"])


Dialogue:
#Person1#: It was a heavy storm last night, wasn't it?
#Person2#: It certainly was. The wind broke several windows. What weather!
#Person1#: Do you know that big tree in front of my house? One of the biggest branches came down in the night.
#Person2#: Really? Did it do any damage to your home?
#Person1#: Thank goodness! It is far away from that.
#Person2#: I really hate storms. It's about time we had some nice spring weather.
#Person1#: It's April, you know. The flowers are beginning to blossom.
#Person2#: Yes, that's true. But I still think the weather is terrible.
#Person1#: I suppose we should not complain. We had a fine March after all.

Reference Summary:
#Person1# and #Person2# are talking about the heavy storm last night, and #Person1#'s positive. #Person2# thinks the weather is terrible. #Person1# thinks they should not complain.

Model Summary GPT:
#Person1# and #Person2# discuss a heavy storm last night. The wind broke several windows and a big tree came down in fro

# Fine Tune

In [ ]:
dfmix = df[['id','dialogue','summary']]

In [ ]:
dfmix

,id,dialogue,summary
0,0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,1,Olivia: Who are you voting for in this electio...,Olivia and Oliver both plan to vote for the Li...
2,2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,3,"Edward: Rachel, I think I'm in ove with Bella....",Rachel abruptly interrupted Edward's confessio...
4,4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,14727,Romeo: You are on my ‘People you may know’ lis...,Romeo reached out to Greta as a potential frie...
14728,14728,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,14729,John: Every day some bad news. Japan will hunt...,John expressed his outrage over Japan's decisi...
14730,14730,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [ ]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(dfmix)

In [ ]:
dataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
val_dataset = dataset_samsum['validation']

In [ ]:
val_dataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})

In [ ]:
import datasets
dd = datasets.DatasetDict({"train":dataset,"validation":val_dataset})

In [ ]:
dd

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer_large(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
    with tokenizer_large.as_target_tokenizer():
        target_encodings = tokenizer_large(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
dataset_samsum_pt = dd.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [ ]:
#dataset_val[0]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_large, model=model_large)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project/Fine_Tuning

/content/drive/MyDrive/Project/Fine_Tuning


In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='bart-samsum-half', num_train_epochs=25, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 
     

In [ ]:
trainer = Trainer(model=model_large, args=trainer_args,
                  tokenizer=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['train'], 
                  eval_dataset=dataset_samsum_pt['validation'])

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.087400,1.478014
1000,0.684100,1.658565
1500,0.413300,1.965265
2000,0.251600,2.348899
2500,0.159100,2.617553
3000,0.089200,2.997341
3500,0.066100,3.016472
4000,0.047300,3.298780
4500,0.032900,3.306062


In [ ]:
model_large.save_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3")
tokenizer_large.save_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3")

NameError: ignored

In [ ]:
tokenizer_large = BartTokenizer.from_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3")
model_large = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3").to(device)

In [ ]:
trainer_args = TrainingArguments(
    output_dir='bart-samsum-half', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=8, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 
     

In [ ]:
trainer = Trainer(model=model_large, args=trainer_args,
                  tokenizer=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['train'], 
                  eval_dataset=dataset_samsum_pt['validation'])

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=115, training_loss=1.0131884035856829, metrics={'train_runtime': 541.0638, 'train_samples_per_second': 27.228, 'train_steps_per_second': 0.213, 'total_flos': 1.0860563227017216e+16, 'train_loss': 1.0131884035856829, 'epoch': 1.0})

In [ ]:
rouge_metric = load_metric('rouge')

<ipython-input-187-90542a62301a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [ ]:
dataset_samsum['test']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'], rouge_metric, trainer.model, tokenizer_large, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'bart-large'] )

100%|██████████| 410/410 [10:07<00:00,  1.48s/it]


,rouge1,rouge2,rougeL,rougeLsum
bart-large,0.011862,0.000259,0.011814,0.011824


In [ ]:
model_check = "philschmid/bart-large-cnn-samsum" 

tokenizer_check = BartTokenizer.from_pretrained(model_check)
model_baseline = BartForConditionalGeneration.from_pretrained(model_check).to(device)

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'], rouge_metric, model_baseline, tokenizer_check, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'bart-baseline'] )

100%|██████████| 410/410 [09:13<00:00,  1.35s/it]


,rouge1,rouge2,rougeL,rougeLsum
bart-baseline,0.01283,0.000262,0.012803,0.012832
